In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import matplotlib.pyplot as plt

print("Libraries imported.")

Libraries imported.


In [5]:
# Define the directory and image parameters
data_dir = 'data'
img_size = 128
batch_size = 32

# Use ImageDataGenerator to load images from the folders
datagen = ImageDataGenerator(
    rescale=1./255,          # Normalize pixel values to be between 0 and 1
    validation_split=0.2     # Use 20% of the images for validation
)

# Create a generator for the training data (80% of images)
train_data = datagen.flow_from_directory(
    data_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary',     # It's a two-class problem (mask or no mask)
    subset='training'
)

# Create a generator for the validation data (20% of images)
val_data = datagen.flow_from_directory(
    data_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

print("Data loaded successfully!")

Found 6043 images belonging to 2 classes.
Found 1510 images belonging to 2 classes.
Data loaded successfully!


In [6]:
model = Sequential([
    # First convolutional layer: finds simple features like edges
    Conv2D(32, (3,3), activation='relu', input_shape=(img_size, img_size, 3)),
    MaxPooling2D(2,2),

    # Second convolutional layer: finds more complex features
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    # Flatten the results to feed into a dense layer
    Flatten(),
    
    # A dense layer for classification
    Dense(128, activation='relu'),
    Dropout(0.5), # Dropout helps prevent overfitting

    # Output layer: 1 neuron with a sigmoid activation for binary output
    Dense(1, activation='sigmoid')
])

model.summary()

C:\Users\varin\anaconda3\envs\pydata-book\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 57600)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       7,372,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,392,449 (28.20 MB)

 Trainable params: 7,392,449 (28.20 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
model.compile(
    optimizer="adam", 
    loss="binary_crossentropy", 
    metrics=["accuracy"]
)
print("Model compiled.")

Model compiled.


In [ ]:
# Train the model for 10 epochs (10 passes over the entire dataset)
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10
)

C:\Users\varin\anaconda3\envs\pydata-book\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
 44/189 ━━━━━━━━━━━━━━━━━━━━ 1:12 497ms/step - accuracy: 0.6220 - loss: 0.8854

C:\Users\varin\anaconda3\envs\pydata-book\lib\site-packages\PIL\Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


114/189 ━━━━━━━━━━━━━━━━━━━━ 35s 475ms/step - accuracy: 0.7141 - loss: 0.6639

In [ ]:
model.save('my_custom_mask_detector.h5')
print("Model saved as my_custom_mask_detector.h5")